In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf

# 현재 TensorFlow가 인식한 GPU 장치 목록 출력
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
import numpy as np
import tensorflow as tf

# GPU 메모리 설정 (메모리 증가 방지)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 데이터 읽기
def read_data(filename):
    with open(filename, 'r', encoding='utf8') as f:
        result = [line.split('\t') for line in f.read().splitlines()]
        return result[1:]  # header 제외

# 파일 경로
#Colab 환경에서 파일 경로를 지정합니다.
#train_tmp = read_data('/content/drive/MyDrive/ratings_train.txt')
#test_tmp = read_data('/content/drive/MyDrive/ratings_test.txt')

# 로컬 환경에서 파일 경로를 지정합니다.
train_tmp = read_data('./ratings_train.txt')
test_tmp = read_data('./ratings_test.txt')

def kor_movie(max_num_words=1000):
    train_x, train_y = zip(*[(x[1], int(x[2])) for x in train_tmp])
    test_x, test_y = zip(*[(x[1], int(x[2])) for x in test_tmp])

    tokenizer = Tokenizer(num_words=max_num_words)
    tokenizer.fit_on_texts(train_x)

    token_train_x = tokenizer.texts_to_sequences(train_x)
    token_test_x = tokenizer.texts_to_sequences(test_x)

    return (token_train_x, list(train_y)), (token_test_x, list(test_y)), tokenizer

print('...전처리하기...')
max_num_words = 5000
maxlen = 100
batch_size = 32

(x_train, y_train), (x_test, y_test), tokenizer = kor_movie(max_num_words)

# y 데이터 변환
y_train = np.array(y_train)
y_test = np.array(y_test)

# 입력 데이터 패딩
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')

# 모델 생성
print('...모델만들기...')
model = Sequential()
model.add(Embedding(max_num_words, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('...학습...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))

score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

...전처리하기...
x_train shape: (150000, 100)
x_test shape: (50000, 100)
...모델만들기...
...학습...
Epoch 1/5
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 269s 56ms/step - accuracy: 0.7393 - loss: 0.4882 - val_accuracy: 0.7846 - val_loss: 0.4211
Epoch 2/5
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 257s 55ms/step - accuracy: 0.7949 - loss: 0.3985 - val_accuracy: 0.7864 - val_loss: 0.4172
Epoch 3/5
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 257s 54ms/step - accuracy: 0.8079 - loss: 0.3738 - val_accuracy: 0.7837 - val_loss: 0.4211
Epoch 4/5
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 259s 55ms/step - accuracy: 0.8162 - loss: 0.3574 - val_accuracy: 0.7828 - val_loss: 0.4250
Epoch 5/5
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 260s 55ms/step - accuracy: 0.8218 - loss: 0.3425 - val_accuracy: 0.7813 - val_loss: 0.4458
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.7844 - loss: 0.4379
Test score: 0.44582897424697876
Test accuracy: 0.7813199758529663


In [ ]:
import tensorflow as tf

model.save("sentiment_analysis_model.keras")

In [5]:
import pickle

# 훈련한 Tokenizer 저장
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Tokenizer가 성공적으로 저장되었습니다.")

Tokenizer가 성공적으로 저장되었습니다.


In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 저장된 모델 불러오기
model = tf.keras.models.load_model("sentiment_analysis_model.keras")

# Tokenizer 새로 생성
tokenizer = Tokenizer()

# 새로운 영화 리뷰 데이터
new_reviews = [
    "이 영화 정말 재밌어요!",
    "완전 최악이었어. 시간 낭비.",
    "배우들 연기 최고! 감동적이었어요.",
    "스토리가 너무 뻔해. 기대 이하.",
    "액션 장면이 멋졌다. 몰입감 최고!",
    "너무 지루해서 중간에 나왔어요.",
    "OST가 좋았어요. 분위기가 완벽했어.",
    "이런 영화가 왜 인기인지 모르겠네.",
    "마지막 반전 대박! 소름 돋았어.",
    "연출이 어색하고 대사가 유치했음."
]

# 텍스트를 Tokenizer로 변환
tokenizer.fit_on_texts(new_reviews)  # 새로운 텍스트에 대해 Tokenizer 학습
new_sequences = tokenizer.texts_to_sequences(new_reviews)

# 패딩 추가 (모델 학습 시 사용한 max_len과 동일해야 함)
max_len = 100  # 기존 모델 학습 시 사용한 max_len 값
new_padded = pad_sequences(new_sequences, maxlen=max_len, padding="post")

# 감성 예측 실행
predictions = model.predict(new_padded)

# 결과 출력
for review, pred in zip(new_reviews, predictions):
    sentiment = "긍정 😊" if pred > 0.5 else "부정 😞"
    print(f"리뷰: {review}\n예측 감성: {sentiment} (확률: {pred[0]:.4f})\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
리뷰: 이 영화 정말 재밌어요!
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 완전 최악이었어. 시간 낭비.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 배우들 연기 최고! 감동적이었어요.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 스토리가 너무 뻔해. 기대 이하.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 액션 장면이 멋졌다. 몰입감 최고!
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 너무 지루해서 중간에 나왔어요.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: OST가 좋았어요. 분위기가 완벽했어.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 이런 영화가 왜 인기인지 모르겠네.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 마지막 반전 대박! 소름 돋았어.
예측 감성: 긍정 😊 (확률: 0.5075)

리뷰: 연출이 어색하고 대사가 유치했음.
예측 감성: 긍정 😊 (확률: 0.5075)



In [8]:
#-*- coding: utf-8 -*-

from konlpy.tag import Okt
from pprint import pprint
import nltk

# SVM 모델 구현을 위한 API (1)
from sklearn.svm import LinearSVC
from nltk.classify.scikitlearn import SklearnClassifier

# 파일에서 데이터 읽어오기
def read_data(filename):
    with open(filename, 'r', encoding='utf8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] #header 제외
    return data
    
train_data = read_data('./ratings_train.txt') 
test_data = read_data('./ratings_test.txt') #id, text, sentiment


# Data 개수 확인
print(len(train_data)) # train_data : 150,000
print(len(train_data[0])) #변수 : id, text, sentiment

print(len(test_data)) # test_data : 50,000
print(len(test_data[0])) #변수 : id, text, sentiment

# KoNLPy 의 트위터 형태소 분석기를 통한 토큰화 (2)
pos_tagger = Okt()

def tokenize(doc):
    return['/'.join(t) for t in pos_tagger.pos(doc,norm=True,stem=True)]

train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]

# 데이터 요약 (3)
tokens = [t for d in train_docs for t in d[0]] #각 문장별로 tokenize되있는 걸 unlist화
print(len(tokens))
text = nltk.Text(tokens, name='NMSC') 

# classification 상위 200개 token만 변수로 사용 (4)
selected_words = [f[0] for f in text.vocab().most_common(200)]
def term_exists(doc):
    return {'exists({})'.format(word): 
    		(word in set(doc)) for word in selected_words}

train_docs = train_docs[:50000] #training시간 단축을 위해 50000개만 사용
    
train_xy = [(term_exists(d),c) for d,c in train_docs] #tf matrix랑 비슷한 개념
test_xy =  [(term_exists(d),c) for d,c in test_docs]

# 서포트 벡터 머신 훈련 및 평가 (5)
print("===========================================================")
print("SVM 분류기 실행")
classif = SklearnClassifier(LinearSVC())
classifier_svm = classif.train(train_xy) 
print(nltk.classify.accuracy(classifier_svm, test_xy))
print("===========================================================")


150000
3
50000
3
2159921
SVM 분류기 실행
0.74594


In [9]:
import pickle

# 학습된 SVM 모델 저장
with open("svm_classifier.pkl", "wb") as f:
    pickle.dump(classifier_svm, f)


In [17]:
import pickle
from konlpy.tag import Okt

# 저장된 SVM 모델 불러오기
with open("svm_classifier.pkl", "rb") as f:
    loaded_classifier = pickle.load(f)

# 형태소 분석기 초기화
pos_tagger = Okt()

def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

# 감성 분석을 위한 영화 리뷰 데이터
new_reviews = [
    "이 영화 정말 재밌어요!",
    "완전 최악이었어. 시간 낭비.",
    "배우들 연기 최고! 감동적이었어요.",
    "스토리가 너무 뻔해. 기대 이하.",
    "액션 장면이 멋졌다. 몰입감 최고!",
    "너무 지루해서 중간에 나왔어요.",
    "OST가 좋았어요. 분위기가 완벽했어.",
    "이런 영화가 왜 인기인지 모르겠네.",
    "마지막 반전 대박! 소름 돋았어.",
    "연출이 어색하고 대사가 유치했음."
]
# 저장된 단어 사전 로드
with open("tokenizer.pkl", "rb") as f:
    selected_words = pickle.load(f)

selected_words = list(tokenizer.word_index.keys())

def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 각 리뷰에 대한 감성 분석 수행
for review in new_reviews:
    tokens = tokenize(review)
    features = term_exists(tokens)
    prediction = loaded_classifier.classify(features)
    sentiment = "긍정 😊" if prediction == '1' else "부정 😞"
    print(f"리뷰: {review}\n예측 감성: {sentiment}\n")


리뷰: 이 영화 정말 재밌어요!
예측 감성: 부정 😞

리뷰: 완전 최악이었어. 시간 낭비.
예측 감성: 부정 😞

리뷰: 배우들 연기 최고! 감동적이었어요.
예측 감성: 부정 😞

리뷰: 스토리가 너무 뻔해. 기대 이하.
예측 감성: 부정 😞

리뷰: 액션 장면이 멋졌다. 몰입감 최고!
예측 감성: 부정 😞

리뷰: 너무 지루해서 중간에 나왔어요.
예측 감성: 부정 😞

리뷰: OST가 좋았어요. 분위기가 완벽했어.
예측 감성: 부정 😞

리뷰: 이런 영화가 왜 인기인지 모르겠네.
예측 감성: 부정 😞

리뷰: 마지막 반전 대박! 소름 돋았어.
예측 감성: 부정 😞

리뷰: 연출이 어색하고 대사가 유치했음.
예측 감성: 부정 😞

